In [59]:
import numpy as np
import pandas as pd

In [50]:
data= pd.read_csv("/content/drive/MyDrive/Science Journal/data.csv")
print(data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

le = LabelEncoder()
y = le.fit_transform(data['Category'])

X_train, X_test, y_train, y_test = train_test_split(data['Message'], data['Category'], test_size=0.2, random_state=42)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_sequence_length = 100
X_train_seq = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_seq = pad_sequences(X_test_seq, maxlen=max_sequence_length)

In [68]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

embedding_size = 100  
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, embedding_size, input_length=max_sequence_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(patience=3, monitor='val_loss', mode='min', restore_best_weights=True)
batch_size = 32
epochs = 20
history = model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=(X_test_seq, y_test), callbacks=[early_stopping])

test_loss, test_accuracy = model.evaluate(X_test_seq, y_test, batch_size=batch_size)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [68]:
inputText = input("Enter the text: ")

sequence = tokenizer.texts_to_sequences([inputText])
sequence = pad_sequences(sequence, maxlen=max_sequence_length)

res = model.predict(sequence)[0][0]

if res >= 0.5:
    print("SPAM")
else:
    print("NOT SPAM")

Enter the text: sdf
1/1 [==============================] - 0s 300ms/step
NOT SPAM
